In [2]:
import os
import asyncio

import pandas as pd
import pyodbc

from reclaim_tiktok.scrapping.scrapper import Scrapper

import nest_asyncio
nest_asyncio.apply()

from dotenv import load_dotenv

load_dotenv()
%load_ext autoreload
%autoreload 2

In [61]:
ms_token =["K6ycnFewW5yhPmBCFOe1PwyDvkXz_faFA6BMw8MmyxdRMxR2QpwiAZsNdHCps1nvHPJPn5R-PuJtlKImgAo08FYbRYLYBK4fL5YLnTtSd2I=", "rX-PrFF0AAPmO2xGm05ms_Kt_kozEvHd7pMM-yqZGhpdTxoF3B_RpNJItWLK6FlGSqLLtzLz1kcw7PmQ6WFNUZeUH-QzV7qHZjZk5E7I3k8cMghLvkAMMkA-0C_W", "e1zeZ9T8jeV1yRcyRid_h9z4TqgI-6F5RWJz_zNSUFKVKwyXFDOjj2Zm0JdqR6qLL3MzsI0U-0bfVBxD_ZXfRIOXewE1Hymiq4Nu6ytHg7c="] #os.environ.get("ms_token", None)

# Retrieve data through API based on hashtas

In [69]:
scrapper = Scrapper(ms_token=ms_token[0])

In [159]:
#load hashtags from csv
hashtags = pd.read_csv("data/classifier/hashtag_list_curated.csv")

In [ ]:
hashtags["Hashtag"]

In [ ]:
videos = []

In [5]:
videos = asyncio.run(scrapper.search_videos_by_hashtags(count=10000, hashtags=hashtags["Hashtag"], videos=videos))

Searching for afd


In [235]:
len(videos)

392

# Process Data

In [ ]:

# Process each video and collect the processed data
processed_videos = [scrapper.process_video(video) for video in scrapper.filter_unique_videos(videos)]



# Load to DB

### Connect to db

In [3]:
driver = os.environ["DB_DRIVER"]
server = os.environ["DB_SERVER"]
database = os.environ["DB_DATABASE"]
username = os.environ["DB_USERNAME"]
password = os.environ["DB_PASSWORD"]
connection_str = f"Driver={driver};Server={server},1433;Database={database};Uid={username};Pwd={password};Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;"


In [68]:
cnxn = pyodbc.connect(connection_str)
cursor = cnxn.cursor()

## Add users to users table

In [7]:
# print all column names of Videos db
cursor.execute("SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = N'Users'")
columns = cursor.fetchall()
for column in columns:
    print(column[0])

id
follower_count
following_count
heart_count
video_count
digg_count
verified
nickname
unique_name_id


In [ ]:
scrapper.add_users_to_db(processed_videos, connection_str=connection_str)

## Add sounds to sounds table

Get all column names of sounds table

In [ ]:
# print all column names of Videos db
cursor.execute("SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = N'Sounds'")
columns = cursor.fetchall()
for column in columns:
    print(column[0])

In [ ]:
scrapper.add_sounds_to_db(processed_videos, connection_str=connection_str)

## Add videos to videos table

Get all column names of videos table

In [ ]:
# print all column names of Videos db
cursor.execute("SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = N'Videos'")
columns = cursor.fetchall()
for column in columns:
    print(column[0])

In [ ]:
# add users to db
scrapper.add_videos_to_db(processed_videos, connection_str=connection_str)

In [191]:
# get SELECT * FROM [dbo].[Videos] WHERE transcript_en IS NULL;
cursor.execute("SELECT * FROM [dbo].[Videos] WHERE transcript_en IS NULL")
videos = cursor.fetchall()
print(len(videos))

8623


## Get more videos based on users in db

In [8]:
# Retrieve all users in the database
cursor.execute("SELECT * FROM [dbo].[Users]")
users = cursor.fetchall()
print(len(users))

2524


In [9]:
# get unique_name_id from Users
cursor.execute("SELECT unique_name_id FROM [dbo].[Users]")
unique_name_ids = cursor.fetchall()
print(len(unique_name_ids)) 

2524


In [10]:
# add the elements as strings to a list 
unique_name_ids = [x[0] for x in unique_name_ids]

In [50]:
videos_by_user = []

In [ ]:
# chunk the unique_name_ids list and loop through them
import random


for i in range(0, len(unique_name_ids)):
    try:
        videos_by_user = asyncio.run(scrapper.search_videos_by_users(count=33, users=unique_name_ids[i:i+15], videos=videos_by_user))
    except Exception as e:
        print(e)
        # random select of ms_token in ms_token list without removing the token from the list
        while True:
            random_idx = random.randint(0, len(ms_token)-1)
            new_ms_token = ms_token[random_idx]
            if new_ms_token != scrapper.ms_token:
                break
        scrapper.update_ms_token(ms_token=new_ms_token)
        #time.sleep(2)
        continue
    print(len(videos_by_user))

In [53]:
len(videos_by_user)

11877

## Process data

In [63]:

# Process each video and collect the processed data
processed_videos_by_user = [scrapper.process_video(video) for video in scrapper.filter_unique_videos(videos_by_user)]

print(len(processed_videos_by_user))

# filter processed_videos_by_user for videos that contain "afd" in the author_name or author_username
afd_videos = [video for video in processed_videos_by_user if "afd" in video["author_name"].lower() or "afd" in video["author_username"].lower()]
print(len(afd_videos))

11069
2473


# Add sounds to database

In [64]:
scrapper.add_sounds_to_db(afd_videos, connection_str=connection_str)

Number of new sounds to be added to database 1536
Sounds added to database successfully


# Add videos to database

In [65]:
scrapper.add_videos_to_db(afd_videos, connection_str=connection_str)

Number of new videos to be added to database 2184
Videos added to database successfully


In [70]:
# get count of videos in db
cursor.execute("SELECT COUNT(*) FROM [dbo].[Videos]")
videos_count = cursor.fetchall()
print(videos_count)

[(11673,)]


In [71]:
# add transcript to db entry with transcript_en IS NULL
cursor.execute("SELECT * FROM [dbo].[Videos] WHERE transcript_en IS NULL")
videos_feteched = cursor.fetchall()
print(len(videos_feteched))

11673


In [ ]:
# on the first row add the transcript_en
# video = videos_feteched[0]
# video_id = video[0]
# transcript_en = "This is the english transcript"
# cursor.execute(f"UPDATE [dbo].[Videos] SET transcript_en = '{transcript_en}' WHERE video_id = {video_id}")